# This is my project for computational intelligence based on kaggle dataset of monkeys.

In [ ]:
# Import packages


import os
from pathlib import Path

import cv2
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import tensorflow as tf
from keras import applications
from keras.utils import np_utils
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.callbacks import ReduceLROnPlateau
from tensorflow.python.keras.layers import (
    Conv2D,
    Activation,
    MaxPooling2D,
    Dropout,
    Flatten,
    Dense, GlobalAveragePooling2D,
)
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline

Set data folders

In [ ]:
data_folder = "data"
training_directory = os.path.join(data_folder, "training", "training")
test_directory = os.path.join(data_folder, "validation", "validation")
labels_file = os.path.join(data_folder, "monkey_labels.txt")

Read labels

In [ ]:
labels_df = pd.read_csv(labels_file)
labels_df = labels_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
labels_df.columns = labels_df.columns.str.strip()
labels = labels_df["Common Name"]

### Read images from files

In [ ]:
def convert_image_to_vector(input_image, size):
    resized_image = cv2.resize(input_image, size)
    return resized_image


def convert_image_to_vector_rgb(input_image, size):
    resized_image = cv2.resize(input_image, size)
    img_rgb = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    return img_rgb


def convert_image_to_vector_cubic(input_image, size):
    resized_image = cv2.resize(input_image, size, interpolation=cv2.INTER_CUBIC)
    return resized_image


def convert_image_to_vector_both(input_image, size):
    resized_image = cv2.resize(input_image, size, interpolation=cv2.INTER_CUBIC)
    img_rgb_cubic = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    return img_rgb_cubic


def normalize(input_image):
    mean, std = input_image.mean(), input_image.std()
    input_image = (input_image - mean) / std
    return input_image


def process_image(file, size):
    image_file = cv2.imread(file)
    image_pixels = convert_image_to_vector_both(image_file, size=size)
    image_pixels = normalize(image_pixels)
    image_label = file.split(os.path.sep)[-2][1]
    return image_pixels, image_label

In [ ]:
IMG_SIZE = 128
size = (IMG_SIZE, IMG_SIZE)

training_images = []
training_images_flattened = []
training_labels = []

for path in Path(training_directory).rglob("*.jpg"):
    image, label = process_image(str(path), size)
    training_images.append(image)
    training_images_flattened.append(image.flatten())
    training_labels.append(label)

test_images = []
test_images_flattened = []
test_labels = []

for path in Path(test_directory).rglob("*.jpg"):
    image, label = process_image(str(path), size)
    test_images.append(image)
    test_labels.append(label)
    test_images_flattened.append(image.flatten())

### Plot data structure

In [ ]:
df = pd.DataFrame()
df["labels"] = training_labels
lab = df["labels"]

counts = lab.value_counts()
counts = pd.DataFrame(counts)
counts.index.astype(int)
counts = counts.sort_index()
counts.reset_index(drop=True, inplace=True)

training_labels_df = pd.DataFrame(labels)
training_labels_df.reset_index(drop=True, inplace=True)
training_labels_df = training_labels_df.merge(counts, left_index=True, right_index=True, how="left")
training_labels_df.set_index("Common Name", inplace=True)
training_labels_df.plot(kind="bar")

In [ ]:
df = pd.DataFrame()
df["labels"] = test_labels
lab = df["labels"]
counts = lab.value_counts()
counts = pd.DataFrame(counts)
counts.index.astype(int)
counts = counts.sort_index()
counts.reset_index(drop=True, inplace=True)
test_labels_df = pd.DataFrame(labels)
test_labels_df.reset_index(drop=True, inplace=True)
test_labels_df = test_labels_df.merge(counts, left_index=True, right_index=True, how="left")
test_labels_df.set_index("Common Name", inplace=True)
test_labels_df.plot(kind="bar")
print(len(test_labels))

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)
test_images_flattened = np.array(test_images_flattened)
training_labels_for_flattened = training_labels

training_images = np.array(training_images)
training_labels = np.array(training_labels)
training_images_flattened = np.array(training_images_flattened)
test_labels_for_flattened = test_labels

num_classes = len(np.unique(training_labels))
label_encoder = LabelEncoder()
training_labels = label_encoder.fit_transform(training_labels)
test_labels = label_encoder.fit_transform(test_labels)
test_labels = np_utils.to_categorical(test_labels, num_classes)
training_labels = np_utils.to_categorical(training_labels, num_classes)

### Test linear and logistic regression using SGD classifier

In [ ]:
sgd_lin = SGDClassifier(max_iter=1000, tol=1e-3)
sgd_lin.fit(training_images_flattened, training_labels_for_flattened)
y_pred = sgd_lin.predict(test_images_flattened)
print(
    'SGD-linear percentage correct: ',
    100*np.sum(y_pred == test_labels_for_flattened)/len(test_labels_for_flattened)
)

In [ ]:
sgd_log = SGDClassifier(loss="log", max_iter=1000, tol=1e-3)
sgd_log.fit(training_images_flattened, training_labels_for_flattened)
y_pred = sgd_log.predict(test_images_flattened)
print(
    'SGD-logistic percentage correct: ',
    100*np.sum(y_pred == test_labels_for_flattened)/len(test_labels_for_flattened)
)

### Test KNN classifier

In [ ]:
model7 = KNeighborsClassifier(n_neighbors=7)
model14 = KNeighborsClassifier(n_neighbors=14)
model2 = KNeighborsClassifier(n_neighbors=2)
model1 = KNeighborsClassifier(n_neighbors=1)

model1.fit(training_images_flattened, training_labels)
model2.fit(training_images_flattened, training_labels)
model7.fit(training_images_flattened, training_labels)
model14.fit(training_images_flattened, training_labels)

acc1 = model1.score(test_images_flattened, test_labels)
acc2 = model2.score(test_images_flattened, test_labels)
acc7 = model7.score(test_images_flattened, test_labels)
acc14 = model14.score(test_images_flattened, test_labels)

print("Accuracy for 1n: {:.2f}%".format(acc1 * 100))
print("Accuracy for 2n: {:.2f}%".format(acc2 * 100))
print("Accuracy for 7n: {:.2f}%".format(acc7 * 100))
print("Accuracy for 14n: {:.2f}%".format(acc14 * 100))

In [ ]:
LR = 1e-3
channels=3
seed=1337
batch_size = 64
num_classes = 10
epochs = 200
data_augmentation = True
num_predictions = 20

# Training generator
train_data_generator = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_data_generator.flow_from_directory(training_directory,
                                                    target_size=size,
                                                    batch_size=batch_size,
                                                    seed=seed,
                                                    shuffle=True,
                                                    class_mode='categorical')

# Test generator
test_data_generator = ImageDataGenerator(rescale=1./255)
validation_generator = test_data_generator.flow_from_directory(test_directory,
                                                  target_size=size,
                                                  batch_size=batch_size,
                                                  seed=seed,
                                                  shuffle=False,
                                                  class_mode='categorical')

### Implement models

In [ ]:
base_resnet_model = applications.resnet50.ResNet50(
    weights=None,
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
x = base_resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
resnet_model = Model(inputs = base_resnet_model.input, outputs = predictions)
resnet_model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)
resnet_model.summary()
keras.utils.plot_model(resnet_model)

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation("softmax"))

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

### Fit models


In [ ]:
EPOCHS = 30

es = EarlyStopping(
    monitor="loss",
    mode="auto",
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor = "loss",
    factor = 0.3,
    patience = 3,
    min_lr = 1e-5,
    mode = "auto",
    verbose = 1
)

In [ ]:
resnet_model.fit(
    train_generator,
    epochs=EPOCHS,
    callbacks=[es]
)

In [ ]:
resnet_model.save(f"ModelResNet{IMG_SIZE}im_genwithout_rl.h5")

In [ ]:
model.fit(
    training_images,
    training_labels,
    epochs=EPOCHS,
    callbacks=[es]
)

In [ ]:
model.save(f"ModelSimple{IMG_SIZE}with_rl.h5")

### Test models

Residual Network model

In [ ]:
print(resnet_model.evaluate(validation_generator, verbose=2))

Simple model

In [ ]:
print("Testing simple")
test_loss, test_acc = model.evaluate(
    test_images,
    test_labels,
    verbose=2
)
print("\nTest accuracy:", test_acc)


### Test on one image

In [ ]:
n = 200
test_image = test_images[n]
test_image = np.expand_dims(test_image, axis=0)
prediction = model.predict(test_image, batch_size=1)
print(labels[np.argmax(prediction)])
print(labels)

Test saved models

In [ ]:
def handle_model(path):
    print(path.name)
    model = tf.keras.models.load_model(path)
    y_true = test_labels_for_flattened.astype(np.int)
    if "im_gen" in str(path.name):
        y_pred = model.predict(validation_generator, verbose=2)
        predictions = np.argmax(y_pred, axis=1)
        print(model.evaluate(validation_generator, verbose=2))
        sns.heatmap(metrics.confusion_matrix(predictions, y_true), annot=True, fmt="d")
        plt.show()
    else:
        y_pred = model.predict(test_images)
        predictions = np.argmax(y_pred, axis=1)
        print(model.evaluate(test_images, test_labels, verbose=2))
        sns.heatmap(metrics.confusion_matrix(predictions, y_true), annot=True, fmt="d")
        plt.show()

In [ ]:
for path in Path("./").rglob("*.h5"):
    if "32" in str(path.name):
        handle_model(path)